**Training notebook can be found [here](https://www.kaggle.com/bipinkrishnan/efficientnet-b2-no-cross-validation-lb-0-921)**

In [ ]:
from PIL import Image
import sys

import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score

import torch
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset

sys.path.append('../input/efficient-net-deps-1/')
from efficientnet_pytorch.model import EfficientNet
from tqdm.notebook import tqdm

In [ ]:
root_path = '../input/ranzcr-clip-catheter-line-classification/'
test_imgs = root_path+ 'test/'

submission = pd.read_csv(root_path+'sample_submission.csv')

In [ ]:
class LoadData(Dataset):
    def __init__(self, df, transform, test=False):
        super().__init__()
        self.df = df
        self.transform = transform
        self.test = test
        
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        if not self.test:
            img = Image.open(train_imgs+row[0]+'.jpg').convert('RGB')
            labels = torch.from_numpy(row[1:-1].astype(np.float32).values)
        
            return self.transform(img), labels 
        else:
            img = Image.open(test_imgs+row[0]+'.jpg').convert('RGB')
            return self.transform(img)
        
    def __len__(self): return len(self.df)

In [ ]:
def build_model(model='effnet'):
    #building the model for transfer learning
    if model == 'effnet':
        effnet = EfficientNet.from_name('efficientnet-b2', num_classes=11)
        return effnet

In [ ]:
BATCH_SIZE = 32
DEVICE = 'cuda'
IMG_SIZE = 256

transform = transforms.Compose([transforms.Resize((IMG_SIZE, IMG_SIZE)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])])

test_ds = LoadData(submission, transform, test=True)
test_dl = DataLoader(test_ds, BATCH_SIZE, shuffle=False)

In [ ]:
#path to your saved best model
model_path = '../input/effnet-model/model_92.pt'
model = build_model().to(DEVICE)
model.load_state_dict(torch.load(model_path, map_location=DEVICE))

In [ ]:
model.eval()
out = []
with torch.no_grad():
    for data in tqdm(test_dl, total=len(test_dl)):
        out.append((model(data.to(DEVICE)).sigmoid()).cpu())

In [ ]:
preds = torch.cat(out).numpy()
solution = pd.DataFrame(preds, columns=submission.columns[1:])
solution['StudyInstanceUID'] = submission['StudyInstanceUID']
solution = solution.reindex(columns=list(submission))
solution.to_csv('submission.csv', index=False)